## Postgres Setup

Set the following environment variables before running this notebook:
- POSTGRES_HOST
- POSTGRES_PORT
- POSTGRES_DB
- POSTGRES_USER
- POSTGRES_PASSWORD

In [2]:
import os

# Set Postgres environment variables
os.environ["POSTGRES_HOST"] = "localhost"  # your host
os.environ["POSTGRES_PORT"] = "5432"       # your port
os.environ["POSTGRES_DB"] = "routes_db" # your database name
os.environ["POSTGRES_USER"] = "postgres"  # your username
os.environ["POSTGRES_PASSWORD"] = "your_password"  # your password

Next we want to establish a connection to make sure our PostGresSQL server is running as intended, we do this using the `psycopg2` library.

In [3]:
import psycopg2

conn = psycopg2.connect(
    host=os.getenv("POSTGRES_HOST"),
    port=os.getenv("POSTGRES_PORT"),
    dbname=os.getenv("POSTGRES_DB"),
    user=os.getenv("POSTGRES_USER"),
    password=os.getenv("POSTGRES_PASSWORD")
)
print("Connected!")
conn.close()

Connected!


Now we want to create the database inside our server, once this connection is established we also want to create a column for labels like finance and healthcare, then a column for utterances.

In [30]:
import os
from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker, declarative_base
from sqlalchemy import Column, String, Integer, Enum
import enum

Base = declarative_base()

class LabelType(enum.Enum):
    FINANCE = "finance"
    HEALTHCARE = "healthcare"

class Utterance(Base):
    __tablename__ = 'utterances'
    id = Column(Integer, primary_key=True, autoincrement=True)
    utterance = Column(String, nullable=False)
    label = Column(Enum(LabelType), nullable=False)

# Build the connection string
pg_uri = f"postgresql+psycopg2://{os.getenv('POSTGRES_USER')}:{os.getenv('POSTGRES_PASSWORD')}@{os.getenv('POSTGRES_HOST')}:{os.getenv('POSTGRES_PORT')}/{os.getenv('POSTGRES_DB')}"
engine = create_engine(pg_uri)
Session = sessionmaker(bind=engine)
session = Session()

# Create table if not exists
Base.metadata.create_all(engine)

session.query(Utterance).delete()
session.commit()

# Sample data to insert
sample_utterances = [
    ("How should I invest my money?", LabelType.FINANCE),
    ("What's the best way to save for retirement?", LabelType.FINANCE),
    ("How do I create a budget?", LabelType.FINANCE),
    ("What are good investment options?", LabelType.FINANCE),
    ("How can I improve my credit score?", LabelType.FINANCE),
    ("What are the symptoms of diabetes?", LabelType.HEALTHCARE),
    ("How do I lower my blood pressure?", LabelType.HEALTHCARE),
    ("What causes chest pain?", LabelType.HEALTHCARE),
    ("Should I get a flu shot?", LabelType.HEALTHCARE),
    ("I have a headache, what should I do?", LabelType.HEALTHCARE),
    ("How do I start investing in stocks?", LabelType.FINANCE),
    ("What's the difference between a 401k and IRA?", LabelType.FINANCE),
    ("How much should I save for emergency fund?", LabelType.FINANCE),
    ("What are the best credit cards for rewards?", LabelType.FINANCE),
    ("How do I calculate my net worth?", LabelType.FINANCE),
    ("What are the symptoms of high blood pressure?", LabelType.HEALTHCARE),
    ("How do I know if I have diabetes?", LabelType.HEALTHCARE),
    ("What causes migraines?", LabelType.HEALTHCARE),
    ("Should I get a mammogram?", LabelType.HEALTHCARE),
    ("What are the warning signs of a heart attack?", LabelType.HEALTHCARE),
]

# Insert the sample data
for utterance, label in sample_utterances:
    new_utterance = Utterance(utterance=utterance, label=label)
    session.add(new_utterance)

session.commit()

Next we can query our database to find all relevant utterances based of the label we created earlier

In [31]:
# Query utterances from PostgreSQL
finance_utterances = session.query(Utterance.utterance).filter(
    Utterance.label == LabelType.FINANCE
).all()

healthcare_utterances = session.query(Utterance.utterance).filter(
    Utterance.label == LabelType.HEALTHCARE
).all()

Using the search we can look through all our utterances we have for each route

In [32]:
print("List of financial utterances:")
for i, utterance in enumerate(finance_utterances, 1):
    print(f"{i}. {utterance[0]}")

List of financial utterances:
1. How should I invest my money?
2. What's the best way to save for retirement?
3. How do I create a budget?
4. What are good investment options?
5. How can I improve my credit score?
6. How do I start investing in stocks?
7. What's the difference between a 401k and IRA?
8. How much should I save for emergency fund?
9. What are the best credit cards for rewards?
10. How do I calculate my net worth?


In [33]:
print("List of healthcare utterances:")
for i, utterance in enumerate(healthcare_utterances, 1):
    print(f"{i}. {utterance[0]}")

List of healthcare utterances:
1. What are the symptoms of diabetes?
2. How do I lower my blood pressure?
3. What causes chest pain?
4. Should I get a flu shot?
5. I have a headache, what should I do?
6. What are the symptoms of high blood pressure?
7. How do I know if I have diabetes?
8. What causes migraines?
9. Should I get a mammogram?
10. What are the warning signs of a heart attack?


Next we can create our `Routes` using the utterances from the database

In [34]:
from semantic_router.route import Route

# Extract just the utterance strings from the query results
finance_route = Route(
    name="finance",
    utterances=[row[0] for row in finance_utterances]
)

healthcare_route = Route(
    name="healthcare", 
    utterances=[row[0] for row in healthcare_utterances]
)

In [35]:
routes = [finance_route, healthcare_route]

Now we need to create a dense encoder for our router, we do this using OpenAI API embedding models

In [105]:
import os
from getpass import getpass
from semantic_router.encoders import OpenAIEncoder

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") or getpass(
    "Enter OpenAI API Key: "
)

encoder = OpenAIEncoder(
    name="text-embedding-3-large", score_threshold=0.5, dimensions=256
)

Now we can put this all into our router initalization

In [106]:
from semantic_router.routers import SemanticRouter

rl = SemanticRouter(encoder=encoder, routes=routes, auto_sync="local")

2025-06-18 11:02:59 - semantic_router.utils.logger - WARNING - base.py:442 - _get_index() - No index provided. Using default LocalIndex.
2025-06-18 11:03:00 - httpx - INFO - _client.py:1025 - _send_single_request() - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-06-18 11:03:00 - httpx - INFO - _client.py:1025 - _send_single_request() - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-06-18 11:03:00 - semantic_router.utils.logger - WARNING - local.py:213 - _write_config() - No config is written for LocalIndex.


We can check the routes one last time to make sure they are correct and have the right utterances

In [108]:
for route in rl.routes:
    print(f"\nRoute: {route.name.upper()}")
    print("Utterances:")
    for i, utterance in enumerate(route.utterances[:3], 1):
        print(f"  {i}. {utterance}")
    print(f"Score threshold: {route.score_threshold}")
    print(f"Total utterances: {len(route.utterances)}")
    print("\n")


Route: FINANCE
Utterances:
  1. How should I invest my money?
  2. What's the best way to save for retirement?
  3. How do I create a budget?
Score threshold: 0.5
Total utterances: 10



Route: HEALTHCARE
Utterances:
  1. What are the symptoms of diabetes?
  2. How do I lower my blood pressure?
  3. What causes chest pain?
Score threshold: 0.5
Total utterances: 10




Next we can feed the router one of the questions to see if it can correctly identify the route

In [118]:
result = rl(text="How much should I save for emergency fund?")

print(f"Route Name: {result.name}")

2025-06-18 11:11:17 - httpx - INFO - _client.py:1025 - _send_single_request() - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Route Name: finance
